In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!git clone -q https://github.com/coleman-zachery/poker_pytorch.git git_download
!rm -rf libs
!mv git_download/libs .
!rm -rf git_download

from libs.constants import RANKS, SUITS
from libs.helpers import card_strings_to_tensor, card_str_to_tuple, classify_poker_hand, poker_hand_label_to_index, reverse_poker_hand_index
from libs.encoders import CardEncoder, PokerHandClassifier
from libs.generators import generate_straight_flush, generate_four_of_a_kind

In [ ]:
!cat libs/helpers.py

In [ ]:
generate_four_of_a_kind()

In [ ]:
generate_straight_flush()

In [25]:
# ---- training set ----
card_strings = [
    ["A♥", "K♦", "Q♣", "J♠", "__"], # high card
    ["A♥", "K♥", "Q♥", "J♥", "X♥"], # royal flush
    ["9♠", "8♠", "7♠", "6♠", "5♠"], # straight flush
    ["3♦", "3♣", "3♥", "3♠", "9♥"], # four of a kind
    ["4♥", "4♠", "4♣", "9♦", "9♥"], # full house
    ["2♣", "5♣", "9♣", "J♣", "K♣"], # flush
    ["6♥", "5♠", "4♦", "3♣", "2♥"], # straight
    ["7♠", "7♥", "7♣", "2♦", "5♥"], # three of a kind
    ["8♦", "8♣", "4♥", "4♠", "K♥"], # two pair
    ["J♣", "J♠", "3♥", "6♦", "9♣"], # one pair
    ["5♥", "X♠", "7♣", "4♦", "2♥"], # high card
    ["5♥", "5♠", "__", "__", "__"], # one pair with 3 empty cards
    ["9♣", "9♦", "4♥", "4♠", "??"], # two pair and 1 ?? card
    ["__", "__", "__", "__", "__"], # empty hand
]


In [26]:
# create tensors and labels
card_tensors = []
labels = []
for hand in card_strings:
    card_tensor = card_strings_to_tensor(hand)
    card_tensors.append(card_tensor)
    card_tuples = [card_str_to_tuple(cs) for cs in hand]
    label = classify_poker_hand(card_tuples)
    labels.append(poker_hand_label_to_index(label))

# get classification embeddings
card_tensors = torch.stack(card_tensors)
labels = torch.tensor(labels, dtype=torch.long)
# print("Card Tensors:\n", card_tensors)
# print("Labels:\n", labels)
encoder = CardEncoder()
classifier = PokerHandClassifier() # ---- choose classifier ----
encoded_cards = encoder(card_tensors.view(-1, 2))
encoded_hands = encoded_cards.view(card_tensors.size(0), 5, -1)
outputs = classifier(encoded_hands)
print("Classifier Outputs:\n", outputs)

[12, 13, 14, 15]
[11, 12, 13, 14, 15]
[6, 7, 8, 9, 10]
[4, 4, 4, 4, 10]
[5, 5, 5, 10, 10]
[3, 6, 10, 12, 14]
[3, 4, 5, 6, 7]
[3, 6, 8, 8, 8]
[5, 5, 9, 9, 14]
[4, 7, 10, 12, 12]
[3, 5, 6, 8, 11]
[6, 6]
[5, 5, 10, 10]
[]
Classifier Outputs:
 tensor([[-0.2348, -0.0287, -0.1227, -0.2072, -0.3211,  0.0300,  0.2411, -0.1542,
         -0.1193, -0.3678,  0.4060],
        [-0.2378,  0.0542, -0.0700, -0.1861, -0.1586,  0.2078,  0.2798, -0.2164,
         -0.0496, -0.0807,  0.1947],
        [-0.1498, -0.0324, -0.0473,  0.0913, -0.0775,  0.2683,  0.1120, -0.1394,
         -0.0253, -0.1151,  0.0373],
        [-0.1342, -0.1898,  0.0123, -0.1340, -0.2112,  0.1310,  0.1171,  0.1681,
         -0.1585, -0.0707,  0.1629],
        [-0.2323,  0.0155,  0.0302, -0.0215, -0.1231,  0.0074,  0.0746, -0.0515,
         -0.1315, -0.3678,  0.3759],
        [-0.2018, -0.1042,  0.0297, -0.1752, -0.3859, -0.0136,  0.1122,  0.0046,
         -0.1160, -0.2595,  0.2404],
        [-0.1796,  0.0689, -0.0607,  0.0642, -0.0287

In [27]:
# ---- train loop ----
def train_loop(model, data_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0
        for batch_cards, batch_labels in data_loader:
            optimizer.zero_grad()
            encoded_cards = encoder(batch_cards.view(-1, 2))
            encoded_hands = encoded_cards.view(batch_cards.size(0), 5, -1)
            outputs = model(encoded_hands)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(data_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Example of setting up a data loader and training
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(card_tensors, labels)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(classifier.parameters(), lr=0.001)
train_loop(classifier, data_loader, criterion, optimizer, num_epochs=100)


Epoch [1/100], Loss: 2.4430
Epoch [2/100], Loss: 2.3980
Epoch [3/100], Loss: 2.3267
Epoch [4/100], Loss: 2.2706
Epoch [5/100], Loss: 2.2420
Epoch [6/100], Loss: 2.1984
Epoch [7/100], Loss: 2.1528
Epoch [8/100], Loss: 2.0890
Epoch [9/100], Loss: 2.0464
Epoch [10/100], Loss: 1.9757
Epoch [11/100], Loss: 1.9372
Epoch [12/100], Loss: 1.8879
Epoch [13/100], Loss: 1.8732
Epoch [14/100], Loss: 1.8017
Epoch [15/100], Loss: 1.7268
Epoch [16/100], Loss: 1.6954
Epoch [17/100], Loss: 1.6499
Epoch [18/100], Loss: 1.5705
Epoch [19/100], Loss: 1.5119
Epoch [20/100], Loss: 1.4842
Epoch [21/100], Loss: 1.4025
Epoch [22/100], Loss: 1.3596
Epoch [23/100], Loss: 1.2791
Epoch [24/100], Loss: 1.2267
Epoch [25/100], Loss: 1.1762
Epoch [26/100], Loss: 1.1441
Epoch [27/100], Loss: 1.0644
Epoch [28/100], Loss: 1.0292
Epoch [29/100], Loss: 1.0047
Epoch [30/100], Loss: 0.9110
Epoch [31/100], Loss: 0.8970
Epoch [32/100], Loss: 0.8516
Epoch [33/100], Loss: 0.8280
Epoch [34/100], Loss: 0.8164
Epoch [35/100], Loss: 0

In [28]:
# ---- function that outputs the softmax probability for each class >= 1.00%, formatted to .00% and sorted largest to smallest % ----
def get_class_probabilities(model, hand_tensor):
    model.eval()
    with torch.no_grad():
        encoded_cards = encoder(hand_tensor.view(-1, 2))
        encoded_hand = encoded_cards.view(1, 5, -1)
        outputs = model(encoded_hand)
        probabilities = F.softmax(outputs, dim=1).squeeze(0)
        significant_probs = {reverse_poker_hand_index(i): prob.item() for i, prob in enumerate(probabilities)}
        sorted_probs = dict(sorted(significant_probs.items(), key=lambda item: item[1], reverse=True))
        formatted_probs = {k: f"{v*100:.2f}%" for k, v in sorted_probs.items()}
        return formatted_probs

In [32]:
# ---- print tensor hands, expected output, and classifier outputs ----
for i in range(card_tensors.size(0)):
    hand_tensor = card_tensors[i].unsqueeze(0)
    expected_label = labels[i].item()
    hand_type = reverse_poker_hand_index(expected_label)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    #predicted_label = torch.argmax(outputs, dim=1).item()
    hand = card_strings[i]
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")

# ---- test to see if the model can correctly predict each royal flush hand ----
royal_flush_hands = [
    ["A♣", "K♣", "Q♣", "J♣", "X♣"],
    ["A♦", "K♦", "Q♦", "J♦", "X♦"],
    ["A♥", "K♥", "Q♥", "J♥", "X♥"],
    ["A♠", "K♠", "Q♠", "J♠", "X♠"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    hand_type = "royal flush"
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"correct label confidence: {hand_type} -- {probabilities[hand_type]}")



hand: ['A♥', 'K♦', 'Q♣', 'J♠', '__']
correct label confidence: high card -- 97.24%

hand: ['A♥', 'K♥', 'Q♥', 'J♥', 'X♥']
correct label confidence: straight flush -- 96.27%

hand: ['9♠', '8♠', '7♠', '6♠', '5♠']
correct label confidence: straight flush -- 96.06%

hand: ['3♦', '3♣', '3♥', '3♠', '9♥']
correct label confidence: four of a kind -- 95.09%

hand: ['4♥', '4♠', '4♣', '9♦', '9♥']
correct label confidence: full house -- 93.89%

hand: ['2♣', '5♣', '9♣', 'J♣', 'K♣']
correct label confidence: flush -- 94.95%

hand: ['6♥', '5♠', '4♦', '3♣', '2♥']
correct label confidence: straight -- 94.44%

hand: ['7♠', '7♥', '7♣', '2♦', '5♥']
correct label confidence: three of a kind -- 94.90%

hand: ['8♦', '8♣', '4♥', '4♠', 'K♥']
correct label confidence: two pair -- 97.96%

hand: ['J♣', 'J♠', '3♥', '6♦', '9♣']
correct label confidence: one pair -- 95.16%

hand: ['5♥', 'X♠', '7♣', '4♦', '2♥']
correct label confidence: high card -- 97.05%

hand: ['5♥', '5♠', '__', '__', '__']
correct label confidenc

In [31]:
royal_flush_hands = [
    ["__", "A♣", "K♦", "Q♥", "J?"],
]

for hand in royal_flush_hands:
    hand_tensor = card_strings_to_tensor(hand).unsqueeze(0)
    encoded_cards = encoder(hand_tensor.view(-1, 2))
    encoded_hand = encoded_cards.view(1, 5, -1)
    outputs = classifier(encoded_hand)
    probabilities = get_class_probabilities(classifier, hand_tensor)
    print(f"\nhand: {hand}")
    print(f"class probabilities: {probabilities}")


hand: ['__', 'A♣', 'K♦', 'Q♥', 'J?']
class probabilities: {'nothing': '42.89%', 'one pair': '18.03%', 'straight': '16.22%', 'two pair': '9.52%', 'full house': '4.20%', 'straight flush': '3.82%', 'flush': '2.03%', 'royal flush': '1.00%', 'three of a kind': '0.92%', 'four of a kind': '0.72%', 'high card': '0.65%'}
